In [ ]:
# import rasterio
# import rasterio.warp
# import rasterio.fill # Might be needed for nodata handling, but often simple replace is enough
import numpy as np
# import geopandas as gpd
# from shapely.geometry import Point
# import pandas as pd
from osgeo import gdal

In [3]:
input_dem_path = '../../Documents/soilense_data/DEM/q909.dem'

In [4]:
dem_dataset = gdal.Open(input_dem_path)

/Users/rex/soilense/venv/lib/python3.13/site-packages/osgeo/gdal.py:330: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [5]:
if dem_dataset:
    # Define output slope file
    output_slope_path = "q909_slope_output.tif"

    # Process DEM to calculate slope
    # Options can be passed as a list of strings, e.g., options=['-p'] for percent slope
    slope_dataset = gdal.DEMProcessing(output_slope_path,
                                       dem_dataset,
                                       "slope",
                                       computeEdges=True)
                                       # Example for percent slope: options=gdal.DEMProcessingOptions(options=['-p']))


    if slope_dataset:
        slope_array = slope_dataset.GetRasterBand(1).ReadAsArray()
        print("Slope calculation successful. Mean slope:", slope_array.mean())

        # Close datasets
        slope_dataset = None
        dem_dataset = None
    else:
        print("Error: Could not process DEM for slope.")
else:
    print(f"Error: Could not open DEM file {input_dem_path}")

Slope calculation successful. Mean slope: -239.50659


In [ ]:
def calculate_slope(dem_array, cell_size):
    """
    Calculates slope from a DEM array.

    Args:
        dem_array (np.ndarray): 2D NumPy array of elevation values.
        cell_size (float): The size of each cell/pixel in the DEM (e.g., meters).

    Returns:
        np.ndarray: 2D NumPy array of slope values in degrees.
    """
    dy, dx = np.gradient(dem_array, cell_size, cell_size)
    slope_rad = np.arctan(np.sqrt(dx**2 + dy**2))
    slope_deg = np.degrees(slope_rad)
    return slope_deg

ModuleNotFoundError: No module named 'richdem'

In [ ]:
def calculate_aspect(dem_array, cell_size):
    """
    Calculates aspect from a DEM array.

    Args:
        dem_array (np.ndarray): 2D NumPy array of elevation values.
        cell_size (float): The size of each cell/pixel in the DEM.

    Returns:
        np.ndarray: 2D NumPy array of aspect values in degrees (0-360).
    """
    dy, dx = np.gradient(dem_array, cell_size, cell_size)
    aspect_rad = np.arctan2(-dy, dx)  # Note: -dy for cartesian to geographic convention
    aspect_deg = np.degrees(aspect_rad)
    # Adjust aspect to 0-360 degrees, North = 0/360
    # Common conversion: 90 - aspect_deg for angles from East counter-clockwise
    # Or, to match common GIS tools (0 North, clockwise):
    aspect_deg = (450 - aspect_deg) % 360 # Adjusts and handles negative results from arctan2
    # A slightly different adjustment was in the source:
    # aspect_deg = np.where(aspect_deg < 0, 90.0 - aspect_deg, 450.0 - aspect_deg) % 360
    # The above is a common adjustment, if dx, dy are considered as East, North respectively
    return aspect_deg

In [ ]:
def calculate_curvature(dem_array, cell_size):
    """
    Calculates a general curvature (Laplacian) from a DEM array.

    Args:
        dem_array (np.ndarray): 2D NumPy array of elevation values.
        cell_size (float): The size of each cell/pixel in the DEM.

    Returns:
        np.ndarray: 2D NumPy array of curvature values.
    """
    # First derivatives
    dy, dx = np.gradient(dem_array, cell_size, cell_size)
    
    # Second derivatives
    d2y_dy, d2x_dy = np.gradient(dy, cell_size, cell_size) # d(dy)/dy, d(dx)/dy
    d2y_dx, d2x_dx = np.gradient(dx, cell_size, cell_size) # d(dy)/dx, d(dx)/dx

    # General curvature (Laplacian)
    # The source used d2x + d2y from a nested gradient call which might be interpreted differently.
    # Typically, Laplacian is d^2z/dx^2 + d^2z/dy^2
    # Using d2x_dx and d2y_dy:
    curvature = d2x_dx + d2y_dy
    return curvature


AttributeError: 'Affine' object has no attribute 'res'

In [ ]:
def calculate_contours(dem_array, cell_x_coords, cell_y_coords, levels):
    """
    Generates contour lines from a DEM array.

    Args:
        dem_array (np.ndarray): 2D NumPy array of elevation values.
        cell_x_coords (np.ndarray): 1D array of x-coordinates for columns.
        cell_y_coords (np.ndarray): 1D array of y-coordinates for rows.
        levels (list or int): Elevation levels for which to generate contours.

    Returns:
        list: A list of NumPy arrays, where each array contains the (x, y)
              vertices of a contour line segment.
    """
    fig, ax = plt.subplots()
    # contour requires X, Y meshgrid for actual coordinates, or assumes pixel indices
    cs = ax.contour(cell_x_coords, cell_y_coords, dem_array, levels=levels)
    plt.close(fig)  # Close plot to avoid display

    contours_data = []
    for i, collection in enumerate(cs.collections):
        level_contours = []
        for path in collection.get_paths():
            level_contours.append(path.vertices)
        contours_data.append({
            'level': cs.levels[i] if isinstance(cs.levels, (list, np.ndarray)) else cs.levels,
            'paths': level_contours
        })
    return contours_data

In [ ]:
def calculate_flow_direction_d8(dem_array):
    """
    Calculates D8 flow direction from a DEM array.

    Args:
        dem_array (np.ndarray): 2D NumPy array of elevation values.

    Returns:
        np.ndarray: 2D NumPy array of flow direction codes.
                    (Codes might be: 1=E, 2=NE, 4=N, 8=NW, 16=W, 32=SW, 64=S, 128=SE,
                     or other conventions like 1-8, or powers of 2).
                     The example uses powers of 2.
    """
    rows, cols = dem_array.shape
    flow_dir = np.zeros_like(dem_array, dtype=np.uint8)
    
    # D8 direction codes (powers of 2, right to bottom-right clockwise)
    # E, SE, S, SW, W, NW, N, NE
    # Codes: 1  2  4  8   16  32  64 128 (GIS standard from ESRI)
    # The source [7] used:
    #   [[128, 64, 32],  (NW, N, NE in its comment mapping)
    #    [  1,  0, 16],  (W, center, E)
    #    [  2,  4,  8]]  (SW, S, SE)
    # This seems to be a non-standard mapping or a typo in the comment.
    # Let's use a common convention:
    # 1=E, 2=NE, 4=N, 8=NW, 16=W, 32=SW, 64=S, 128=SE
    # Offsets: (row, col), Code
    # (0,1)=E=1, (-1,1)=NE=2, (-1,0)=N=4, (-1,-1)=NW=8, 
    # (0,-1)=W=16, (1,-1)=SW=32, (1,0)=S=64, (1,1)=SE=128
    
    # Corrected D8 directions and codes (ESRI convention)
    # dx, dy, code
    d_lookup = [
        (0, 1, 1),   # E
        (-1, 1, 2),  # NE
        (-1, 0, 4),  # N
        (-1, -1, 8), # NW
        (0, -1, 16), # W
        (1, -1, 32), # SW
        (1, 0, 64),  # S
        (1, 1, 128)  # SE
    ]

    for r in range(1, rows - 1):
        for c in range(1, cols - 1):
            center_elev = dem_array[r, c]
            max_drop = 0
            best_dir = 0 # Default for flat areas/pits

            for dr, dc, d_code in d_lookup:
                nr, nc = r + dr, c + dc
                neighbor_elev = dem_array[nr, nc]
                drop = (center_elev - neighbor_elev) # Drop is positive if flowing down
                
                # For D8, typically uses drop / distance for true slope
                # Simple D8 often just uses raw elevation difference
                # distance = np.sqrt(dr**2 + dc**2) # 1 for cardinal, sqrt(2) for diagonal
                # current_slope = drop / distance if distance > 0 else 0
                
                # Using simple drop as per source [7] logic structure
                if drop > max_drop:
                    max_drop = drop
                    best_dir = d_code
            
            flow_dir[r, c] = best_dir
    return flow_dir